# Ridge Regression

In [61]:
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file
from scipy import sparse
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error


In [62]:
%pwd
%cd Downloads


[Errno 2] No such file or directory: 'Downloads'
/Users/dlaldea/Downloads


### Solve the ridge regression problem on the training set:


In [82]:
np.random.seed(42)
def Read_Data_SVM(filename):
    data=load_svmlight_file(filename)
    X=data[0]
    y=data[1].reshape(-1,1)
    return X,y

X,y=Read_Data_SVM('cpusmall_scale.txt')
y=preprocessing.maxabs_scale(y)

In [83]:
def Split_Data(y,X, test_ratio):
    n=X.shape
    indices=np.random.permutation(n[0])
    test_set_size=int(n[0]*test_ratio)
    test_indices=indices[:test_set_size]
    train_indices=indices[test_set_size:]
    return X[train_indices,:], X[test_indices,:], y[train_indices], y[test_indices]

training, testing, y_train, y_test=Split_Data(y,X,0.2)

In [84]:
#Ridge Regression on Training Set
def Ridge_Reg(training, testing, y_test, y_train, L):
    n = np.shape(training)
    theta_hat=np.random.rand(n[1],1)#creates a p-row*1-col
    w=np.linalg.inv(np.transpose(training).dot(training)+L*np.eye(n[1])).dot(np.transpose(training).dot(y_train))
    #Compute MSE on testing set
    m=np.shape(testing)
    test_pred=testing.dot(w)
    mse=mean_squared_error(test_pred,y_test)
    
    print("mse for lambda=",L, "is:",round(mse,4))

Ridge_Reg(training, testing, y_test, y_train, 1)


('mse for lambda=', 1, 'is:', 0.0092)


### Run the Ridge Regression for multiple lambdas and report the test MSE for each 

In [85]:
L_vec=[0.01,0.1,1,10,100]

for L in L_vec:
    Ridge_Reg(training, testing, y_test, y_train, L)
    

('mse for lambda=', 0.01, 'is:', 0.0092)
('mse for lambda=', 0.1, 'is:', 0.0092)
('mse for lambda=', 1, 'is:', 0.0092)
('mse for lambda=', 10, 'is:', 0.0094)
('mse for lambda=', 100, 'is:', 0.011)


## Bonus Ridge Regression with Scikit Learn

In [86]:
from sklearn.linear_model import Ridge
ridge_reg=Ridge(alpha=1, solver="cholesky",fit_intercept=False)
ridge_reg.fit(training, y_train)
yr=ridge_reg.predict(testing)
print round(sklearn.metrics.mean_squared_error(y_test,yr),4) 

#Problem 1.1 and 1.2 Try for multiple lambdas:
L_vec=[0.01,0.1,1,10,100]
for L in L_vec:
    ridge_reg=Ridge(alpha=L, solver="cholesky", fit_intercept=False)
    ridge_reg.fit(training, y_train)
    yr=ridge_reg.predict(testing)
    mse=round(sklearn.metrics.mean_squared_error(y_test,yr),4)
    print ("MSE for lambda=",L,"is",mse)


0.0092
('MSE for lambda=', 0.01, 'is', 0.0092)
('MSE for lambda=', 0.1, 'is', 0.0092)
('MSE for lambda=', 1, 'is', 0.0092)
('MSE for lambda=', 10, 'is', 0.0094)
('MSE for lambda=', 100, 'is', 0.011)


### Problem 1.3 & 1.4 given a stepsize report MSE with E2006 data

In [97]:
np.random.seed(42)
train_E2006, yE2006_train=Read_Data_SVM("E2006.train")
test_E2006, yE2006_test=Read_Data_SVM("E2006.test")


In [98]:
train_E2006=train_E2006[:,0:150358]
train_scaled=preprocessing.maxabs_scale(train_E2006)
test_scaled=preprocessing.maxabs_scale(test_E2006)

ytrain_scaled=preprocessing.maxabs_scale(yE2006_train)
ytest_scaled=preprocessing.maxabs_scale(yE2006_test)

n=train_scaled.shape
theta_hat=np.random.rand(n[1],1)


In [99]:
def gradient(x_t,x,theta,y,L):
    y_pred=x.dot(theta)
    g=x_t.dot(y_pred-y)+L*theta
    return g
    

In [100]:
def StepWise_G(w,train, y, step, e):
    L=1
    train_t=np.transpose(train)
    r0=np.linalg.norm(gradient(train_t,train,w,y,L))
    for i in range(0,50):
        g=gradient(train_t,train,w,y,L)
        g_norm=np.linalg.norm(g)
        if g_norm <= e*r0:
            print(g_norm)
            break
        w=w-step*g
    return w

In [101]:
step_vec=[1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
m=test_scaled.shape
for step in step_vec:
    theta_hat=np.random.rand(n[1],1)
    w=StepWise_G(theta_hat,train_scaled,ytrain_scaled,step,0.001)
    y_pred=test_scaled.dot(w)
    mse=round(mean_squared_error(ytest_scaled,y_pred),3)
    #accuracy=accuracy_score(ytest_scaled, y_pred)
    print("MSE_testing for step=", step, "is", mse)


('MSE_testing for step=', 1e-07, 'is', 3577.604)
('MSE_testing for step=', 1e-06, 'is', 506.091)
('MSE_testing for step=', 1e-05, 'is', 296.667)
('MSE_testing for step=', 0.0001, 'is', 8.383984832197233e+99)
('MSE_testing for step=', 0.001, 'is', 1.0062698705286456e+204)
('MSE_testing for step=', 0.01, 'is', 2.531266086469879e+304)


In [102]:
#try with non-scaled data
step_vec=[1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
m=test_E2006.shape
for step in step_vec:
    theta_hat=np.random.rand(n[1],1)
    w=StepWise_G(theta_hat,train_E2006,yE2006_train,step,0.001)
    y_pred=test_E2006.dot(w)
    mse=round(mean_squared_error(yE2006_test,y_pred),3)
    #accuracy=accuracy_score(ytest_scaled, y_pred)
    print("MSE_testing for step=", step, "is", mse)


('MSE_testing for step=', 1e-07, 'is', 1.135)
('MSE_testing for step=', 1e-06, 'is', 0.166)
('MSE_testing for step=', 1e-05, 'is', 0.163)
('MSE_testing for step=', 0.0001, 'is', 2.401810359311467e+126)
('MSE_testing for step=', 0.001, 'is', 3.3555075473468818e+227)
('MSE_testing for step=', 0.01, 'is', inf)


### Bonus: Ridge Gradient Descent for E2006 data

In [104]:
from sklearn.linear_model import SGDRegressor 
#Problem 1.3 and 1.4 with scaled data
step_vec=[1e-7,1e-6, 1e-5, 1e-4,1e-3]
for step in step_vec:
#Using Stochastic Gradient Descent
    sgd_reg=SGDRegressor(penalty="l2", alpha=1,fit_intercept=False,n_iter=50, eta0=step)
    sgd_reg.fit(train_scaled, ytrain_scaled.ravel())
    yr_SGD=sgd_reg.predict(test_scaled)
    mse=round(mean_squared_error(ytest_scaled,yr_SGD),4)
    print("MSE for learning_rate=",step,"is:",mse)

('MSE for learning_rate=', 1e-07, 'is:', 0.2791)
('MSE for learning_rate=', 1e-06, 'is:', 0.1902)
('MSE for learning_rate=', 1e-05, 'is:', 0.019)
('MSE for learning_rate=', 0.0001, 'is:', 0.0115)
('MSE for learning_rate=', 0.001, 'is:', 0.0115)


In [105]:
from sklearn.linear_model import SGDRegressor 
#Problem 1.3 and 1.4 with non-scaled data
step_vec=[1e-7,1e-6, 1e-5, 1e-4,1e-3]
for step in step_vec:
#Using Stochastic Gradient Descent
    sgd_reg=SGDRegressor(penalty="l2", alpha=1,fit_intercept=True,n_iter=50, eta0=step)
    sgd_reg.fit(train_E2006, yE2006_train.ravel())
    yr_SGD=sgd_reg.predict(test_E2006)
    mse=round(mean_squared_error(yE2006_test,yr_SGD),4)
    print("MSE for learning_rate=",step,"is:",mse)

('MSE for learning_rate=', 1e-07, 'is:', 13.6581)
('MSE for learning_rate=', 1e-06, 'is:', 6.4144)
('MSE for learning_rate=', 1e-05, 'is:', 0.2118)
('MSE for learning_rate=', 0.0001, 'is:', 0.1944)
('MSE for learning_rate=', 0.001, 'is:', 0.193)


## Problem 2: Classification (Logistic Regression)

In [106]:
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file
from scipy import sparse
from sklearn.metrics import accuracy_score

np.random.seed(42)

In [107]:
%cd Downloads

[Errno 2] No such file or directory: 'Downloads'
/Users/dlaldea/Downloads


In [108]:
def Read_Data_SVM(filename):
    data=load_svmlight_file(filename)
    X=data[0]
    y=data[1].reshape(-1,1)
    return X,y

In [109]:
def Split_Data(y,X, test_ratio):
    n=X.shape
    indices=np.random.permutation(n[0])
    test_set_size=int(n[0]*test_ratio)
    test_indices=indices[:test_set_size]
    train_indices=indices[test_set_size:]
    return X[train_indices,:], X[test_indices,:], y[train_indices], y[test_indices]

In [122]:
#Problem 2.2
news_X, news_y=Read_Data_SVM('news20.binary')

In [123]:
X_train, X_test, y_train, y_test = Split_Data(news_y, news_X,0.2)

In [116]:
X_train=preprocessing.maxabs_scale(X_train)
X_test=preprocessing.maxabs_scale(X_test)


In [112]:
n=X_train.shape
theta=np.random.rand(n[1],1)

In [113]:
def Gradient_Log(y,w,xty, L):
    e=np.exp(w.T.dot(xty))
    g=(-xty/(1+e))-L*w
    return g

In [114]:
def SGD_Log(w,train, y, step, e):
    L=1
    xty=train.T.dot(y)
    r0=np.linalg.norm(Gradient_Log(y,w,xty,L))
    for i in range(0,50):
        g=Gradient_Log(y,w,xty,L)
        g_norm=np.linalg.norm(g)
        if g_norm <= e*r0:
            return w
            break
        w=w-step*g
    return w

In [117]:
step_vec=[1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1,1]
for step in step_vec:
    w=SGD_Log(theta,X_train,y_train,step,0.001)
    w_pred=X_test.dot(w)
    labels= map(lambda label: -1 if np.sign(label)== -1 else 1, w_pred)
    accuracy=round(accuracy_score(y_test, labels),4)
    print("Accuracy for step=", step, "is",accuracy)

('Accuracy for step=', 1e-07, 'is', 0.5004)
('Accuracy for step=', 1e-06, 'is', 0.5004)
('Accuracy for step=', 1e-05, 'is', 0.5004)
('Accuracy for step=', 0.0001, 'is', 0.5004)
('Accuracy for step=', 0.001, 'is', 0.5024)
('Accuracy for step=', 0.01, 'is', 0.5076)
('Accuracy for step=', 0.1, 'is', 0.8597)
('Accuracy for step=', 1, 'is', 0.804)


### Bonus: SGD for Logistic using Scikit Learn


In [118]:
from sklearn.linear_model import SGDClassifier
step_vec=[1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
for step in step_vec:
#with fixed step-size for alpha=1 and different step
    clf=SGDClassifier(loss="log", penalty="l2", alpha=1,eta0=step,n_iter=50, fit_intercept=False)
    clf.fit(X_train, y_train.ravel())
    yr=clf.predict(X_test)
    accuracy=round(accuracy_score(y_test,yr),4)
    print("accuracy for lambda=1 and step=",step,"is",accuracy)

('accuracy for lambda=1 and step=', 1e-06, 'is', 0.8132)
('accuracy for lambda=1 and step=', 1e-05, 'is', 0.8132)
('accuracy for lambda=1 and step=', 0.0001, 'is', 0.8132)
('accuracy for lambda=1 and step=', 0.001, 'is', 0.8132)
('accuracy for lambda=1 and step=', 0.01, 'is', 0.8132)
('accuracy for lambda=1 and step=', 0.1, 'is', 0.8132)


### Problem 2.3 Line Search 

In [119]:
def f(w,L,xty):
    f=(np.log(1+np.exp(-w.T.dot(xty)))+(L/2)*np.linalg.norm(w)**2)
    return f
    

In [120]:
#Gradient Descent with line search 

def LineGDesc(x,y,w,e,L):
    xty=x.T.dot(y) #p*1
    r0=np.linalg.norm(Gradient_Log(y,w,xty,L))
    for i in range(0,50):
        g=Gradient_Log(y,w,xty,L)
        g_norm=np.linalg.norm(g)
        step=1
        if g_norm <=e*r0:
            break
        else: 
        #We define the fist instances
            b=True
            while b:
                #We update for each step wht new_f is 
                w_test=w-step*g
                if f(w_test,L,xty)>= f(w, L, xty):
                    step=step/2.0
                else: 
                    b=False
                if step <= 1e-5:
                    break
            w=w_test-step*g
    return w
    

In [124]:
L=[1e-6, 1e-5, 1e-4, 1e-3, 1e-2,1e-1]
theta=np.random.rand(n[1],1)
for l in L:
    w=LineGDesc(X_train,y_train,theta,0.001,l)
    w_pred=X_test.dot(w)
    labels=map(lambda label: -1 if np.sign(label) == -1 else 1, w_pred)
    accuracy=round(accuracy_score(y_test, labels),4)
    print("Accuracy for lambda=",l , "is",accuracy)

('Accuracy for lambda=', 1e-06, 'is', 0.8005)
('Accuracy for lambda=', 1e-05, 'is', 0.8005)
('Accuracy for lambda=', 0.0001, 'is', 0.8005)
('Accuracy for lambda=', 0.001, 'is', 0.8005)
('Accuracy for lambda=', 0.01, 'is', 0.8007)
('Accuracy for lambda=', 0.1, 'is', 0.8037)
